# Part 1: Generate Base Images and CSV Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import os
import csv
#TODO: Change the example usage to python args
# Example usage
start_idx = 0
end_idx = 2

# Directories names
base_dir = 'image'
csv_dir = 'csv'

In [2]:
# Create the output directory if it doesn't exist
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

# Create the output directory if it doesn't exist
if not os.path.exists(os.path.join(base_dir, 'defect')):
    os.makedirs(os.path.join(base_dir, 'defect'))

# Create the output directory if it doesn't exist
if not os.path.exists(os.path.join(base_dir, 'healthy')):
    os.makedirs(os.path.join(base_dir, 'healthy'))

def EnclosedCurve(r_random, N_angle, H, Center_X, Center_Y):
    ang = np.linspace(0, 2*np.pi/N_angle*(N_angle-1), N_angle)

    rho = np.random.rand(H) * 0.15 * np.logspace(-1.5, -2.5, H)
    phi = np.random.rand(H) * 2 * np.pi

    r = np.zeros(N_angle) + r_random

    for h in range(H):
        r = r + rho[h] * np.sin(h * ang + phi[h])

    x = r * np.cos(ang) + Center_X
    y = r * np.sin(ang) + Center_Y

    return x, y, r

def generate_random_array(N, resolution, min_value, max_value):
    random_array = []
    while len(random_array) < N:
        random_value = min_value + (max_value - min_value) * rd.random()
        rounded_value = round(random_value / resolution) * resolution
        random_array.append(rounded_value)
    return random_array

def save_image(filename, r_max, x_trunk_L1, y_trunk_L1, x_trunk_L2, y_trunk_L2, x_trunk_L3, y_trunk_L3, x_Bcavity, y_Bcavity, layer1, layer2, layer3, cavity_colour):
    plt.figure(figsize=(10, 10))
    plt.fill(x_trunk_L1, y_trunk_L1, color=layer1, linestyle='none')
    plt.fill(x_trunk_L2, y_trunk_L2, color=layer2, linestyle='none')
    plt.fill(x_trunk_L3, y_trunk_L3, color=layer3, linestyle='none')
    plt.fill(x_Bcavity, y_Bcavity, color=cavity_colour, linestyle='none')
    plt.xlabel('x(t)')
    plt.ylabel('y(t)')
    plt.axis([-r_max, r_max, -r_max, r_max])

    # circle = plt.Circle((0, 0), r_max, color='white', fill=False, linestyle='solid', linewidth=1)
    # plt.gca().add_patch(circle)
    
    # Set the aspect ratio to be equal
    plt.gca().set_aspect('equal')
    # Apply tight layout
    plt.tight_layout()
    plt.axis('off')
    plt.savefig(filename, format='png', dpi=300, bbox_inches='tight', transparent=True, pad_inches=0, pil_kwargs={'icc_profile': None})
    plt.close()
    
N_trunk = end_idx - start_idx

# Set parameters
Rmin_trunk = 0.15
Rmax_trunk = 0.30
CenTrunk_X = 0
CenTrunk_Y = 0
N_angle = 360
safety_d = 0.01

# Small cavity, if not needed can be removed
Rmin_Scavity = 0.03
Rmax_Scavity = 0.09

layer1 = [1, 0.875, 0]
layer2 = [1, 0.625, 0]
layer3 = [1, 0.375, 0]
cavity_colour = [1, 0.125, 0]

# Initialize arrays to store data
x_trunk_L1 = np.zeros((N_trunk, N_angle))
y_trunk_L1 = np.zeros((N_trunk, N_angle))
x_trunk_L2 = np.zeros((N_trunk, N_angle))
y_trunk_L2 = np.zeros((N_trunk, N_angle))
x_trunk_L3 = np.zeros((N_trunk, N_angle))
y_trunk_L3 = np.zeros((N_trunk, N_angle))
x_Bcavity = np.zeros((N_trunk, N_angle))
y_Bcavity = np.zeros((N_trunk, N_angle))
r_trunk_L3 = np.zeros((N_trunk, N_angle))
r_trunk_L2 = np.zeros((N_trunk, N_angle))
r_trunk_L1 = np.zeros((N_trunk, N_angle))
r_Bcavity = np.zeros((N_trunk, N_angle))
hole_factor = np.zeros(N_trunk)
R_center_Bcavity = np.zeros(N_trunk)
theta_Bcavity = np.zeros(N_trunk)
CenBcavity_X = np.zeros(N_trunk)
CenBcavity_Y = np.zeros(N_trunk)

# Up: upperbound, IS: conductive layer, HW: heartwood
up_IS = 0.9
lo_IS = 0.75
up_bark = 1.1
lo_bark = 1.05

radius = generate_random_array(N_trunk, 0.002, 0.2, 0.4)
eps_bark = generate_random_array(N_trunk, 0.1, 2, 4)
eps_sapwood = generate_random_array(N_trunk, 0.1, 2, 4)
eps_heartwood = generate_random_array(N_trunk, 0.1, 2, 4)
trunk_dist = generate_random_array(N_trunk, 0.002, 0.5, 0.6)
eps_decay = generate_random_array(N_trunk, 0.1, 20, 30)

for i in range(N_trunk):
    H = 8
    R_rand_trunk = np.random.rand(1)*(Rmax_trunk-Rmin_trunk) + Rmin_trunk
    x_trunk_L2[i], y_trunk_L2[i], r_trunk_L2[i] = EnclosedCurve(R_rand_trunk, N_angle, 13, CenTrunk_X, CenTrunk_Y)

    ratio_L3 = np.random.rand()*(up_IS-lo_IS) + lo_IS
    x_trunk_L3[i] = ratio_L3 * x_trunk_L2[i]
    y_trunk_L3[i] = ratio_L3 * y_trunk_L2[i]
    r_trunk_L3[i] = ratio_L3 * r_trunk_L2[i]

    ratio_L1 = np.random.rand()*(up_bark-lo_bark) + lo_bark
    x_trunk_L1[i] = ratio_L1 * x_trunk_L2[i]
    y_trunk_L1[i] = ratio_L1 * y_trunk_L2[i]
    r_trunk_L1[i] = ratio_L1 * r_trunk_L2[i]

    Rmin_trunk_L3 = np.min(r_trunk_L3[i])
    Rmax_Bcavity = 0.8 * Rmin_trunk_L3
    Rmin_Bcavity = 0.05
    hole_factor[i] = np.random.rand(1)
    R_rand_Bcavity = hole_factor[i] * (Rmax_Bcavity - Rmin_Bcavity) + Rmin_Bcavity

    min_d_Bcavity = 0.02
    max_d_Bcavity = Rmin_trunk_L3 - R_rand_Bcavity - safety_d
    R_center_Bcavity[i] = min_d_Bcavity + np.random.rand() * (max_d_Bcavity - min_d_Bcavity)
    theta_Bcavity[i] = 2 * np.pi * np.random.rand()
    CenBcavity_X[i] = R_center_Bcavity[i] * np.cos(theta_Bcavity[i])
    CenBcavity_Y[i] = R_center_Bcavity[i] * np.sin(theta_Bcavity[i])

    shape_factor = np.random.rand()
    max_shape = 20
    min_shape = 15
    shape_cavity = round(shape_factor * (max_shape - min_shape) + min_shape)
    x_Bcavity[i], y_Bcavity[i], r_Bcavity[i] = EnclosedCurve(R_rand_Bcavity, N_angle, shape_cavity, CenBcavity_X[i], CenBcavity_Y[i])

    save_image('image/defect/defect{}.png'.format(i),np.max(r_trunk_L1[i]), x_trunk_L1[i], y_trunk_L1[i], x_trunk_L2[i], y_trunk_L2[i], x_trunk_L3[i], y_trunk_L3[i], x_Bcavity[i], y_Bcavity[i], layer1, layer2, layer3, cavity_colour)
    save_image('image/healthy/healthy{}.png'.format(i),np.max(r_trunk_L1[i]), x_trunk_L1[i], y_trunk_L1[i], x_trunk_L2[i], y_trunk_L2[i], x_trunk_L3[i], y_trunk_L3[i], np.zeros((1, N_angle)), np.zeros((1, N_angle)), layer1, layer2, layer3, cavity_colour)
    # print(CenBcavity_X[i], CenBcavity_Y[i], np.mean(r_Bcavity[i]), trunk_dist[i], radius[i], eps_bark[i], eps_sapwood[i], eps_heartwood[i], eps_decay[i])


In [3]:
def mean_row(arr):
    # Use numpy's max function along axis 1 to get the maximum of each row
    max_values = np.mean(arr, axis=1)
    return max_values

def save_arrays_to_csv(file_path, *arrays):
    # Transpose the arrays to create rows from the corresponding elements
    rows = zip(*arrays)
    
    # Open the CSV file in write mode
    with open(file_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        
        # Write each row to the CSV file
        for row in rows:
            writer.writerow(row)

idx = [i for i in range(start_idx, end_idx)]
r_cavity = mean_row(r_Bcavity)

file_path = f'{csv_dir}/data_{start_idx}_{end_idx}.csv'

# Call the function to save the arrays to the CSV file (it will overwrite the file if it exists)
save_arrays_to_csv(file_path, idx, CenBcavity_X, CenBcavity_Y, r_cavity, trunk_dist, radius, eps_bark, eps_sapwood, eps_heartwood, eps_decay)


# Part 2: Generate required files for each idx, mode

In [4]:
import os
import pandas as pd
import numpy as np
import math
from PIL import Image
import h5py
from gprMax.gprMax import api
from tools.outputfiles_merge import merge_files
import glob
import shutil
import matplotlib.pyplot as plt

#TODO: Change the example usage to loops, start and stop in python
# Example usage:
idx = 0
mode = 'defect'

# Directories Names
working_dir = 'working'
output_dir = 'output'
csv_path = "csv"

# Specify the headers as a list
headers = ["idx", "CenBcavity_X", "CenBcavity_Y", "r_cavity", "trunk_dist", "radius", "eps_bark", "eps_sapwood", "eps_heartwood", "eps_decay"]

In [5]:
# Function to find the folder and CSV file for a given number
def find_csv_for_number(number, folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)

    # Iterate through the files to find the matching CSV file
    for file in files:
        if file.startswith("data_") and file.endswith(".csv"):
            try:
                # Extract the lower and upper bounds from the filename
                _, lower, upper = file.split("_")
                lower = int(lower)
                upper = int(upper.split(".")[0])  # Remove ".csv" extension

                # Check if the given number is within the range
                if lower <= number <= upper:
                    # Build the full path to the CSV file
                    csv_file_path = os.path.join(folder_path, file)
                    return csv_file_path
            except ValueError:
                continue  # Skip files with invalid naming

    # Return None if no matching CSV file is found
    return None

csv_file_path = find_csv_for_number(idx, csv_path)

if csv_file_path:
    # Now you can read the CSV file using your preferred method (e.g., pandas)
    df = pd.read_csv(csv_file_path, header=None, names=headers)
else:
    print(f"No CSV file found for number {idx}")

df.head()


,idx,CenBcavity_X,CenBcavity_Y,r_cavity,trunk_dist,radius,eps_bark,eps_sapwood,eps_heartwood,eps_decay
0,0,0.028283,-0.001646,0.129566,0.532,0.230,3.0,3.2,2.4,24.6
1,1,-0.012392,-0.018339,0.104531,0.574,0.264,3.7,2.2,3.9,23.2


In [6]:
# Create the output directory if it doesn't exist
if not os.path.exists(working_dir):
    os.makedirs(working_dir)
# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    os.makedirs(os.path.join(output_dir, 'png'))
    os.makedirs(os.path.join(output_dir, 'geometry'))
    os.makedirs(os.path.join(output_dir, 'out'))

H = df.loc[df['idx'] == idx, 'trunk_dist'].values[0] # TODO: read from csv
res = int(np.round((df.loc[df['idx'] == idx, 'radius'].values[0]/0.002 * 2))) #TODO: read from csv

# png gen
target = f'{mode}{idx}'
input_image_path = f'image/{mode}/{target}.png'
png_output_dir = 'png'

L = 1
count = 3

# h5 gen
print(f"Trunk area: {res} x {res} cells")
h5_input_dir = 'png'
h5_output_dir = 'h5'

Trunk area: 230 x 230 cells


In [7]:
# Function to find the angle based on the currentPosition and L
def find_angle(current_position, L, H):
    if current_position < L:
        L3 = L - current_position
        current_angle = math.atan(H / L3) * 180 / math.pi
    elif current_position > L:
        L3 = current_position - L
        current_angle = (math.atan(H / L3)) * 180 / math.pi
        current_angle = 180 - current_angle
    else:  # exactly middle
        current_angle = 90

    return 90 - current_angle

# Function to generate rotated images
def generate_rotated_images(input_image_path, output_directory, target, L, H, count):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    mid = L / 2
    step = (L / (count - 1))

    # Open the original image
    original_image = Image.open(input_image_path)
    dist = []
    rotated_images = []
    for i in range(count):
        dist.append(math.sqrt(H**2 + (mid - i * step)**2))
        angle = find_angle(i * step, mid, H)

        # Rotate the original image by the calculated angle
        rotated_image = original_image.rotate(angle)
        rotated_images.append(rotated_image)

    # Save the rotated image with the angle in the filename
    # output_image_path = os.path.join(output_directory, f'{target}_{i}.png')
    # rotated_image.save(output_image_path)
    for i, rotated_image in enumerate(rotated_images):
        output_image_path = os.path.join(output_directory, f'{target}_{i}.png')
        rotated_image.save(output_image_path)

    dist = np.array(dist)
    rd_dist = np.round(dist / 0.002) * 0.002
    rd_dist = [(round(x, len(str(0.002).split('.')[1]))) for x in rd_dist]
    return rd_dist

pos = generate_rotated_images(input_image_path, png_output_dir, target, L, H, count)
np.array(pos)

array([0.73 , 0.532, 0.73 ])

In [8]:
# Color map
color_map = {
    (0, 0, 0, 0): -1,
    (255, 223, 0, 255): 0,  # Yellow
    (255, 159, 0, 255): 1,  # Yellow
    (255, 95, 0, 255): 2,  # Yellow
    (255, 31, 0, 255): 3  # Red
}

# Function to check if a pixel color is approximately equal to a given color
def is_color_approx(pixel_color, target_color, tolerance=10):
    return np.all(np.abs(pixel_color - target_color) <= tolerance, axis=-1)

# Iterate through all files in the input directory
for image_name in os.listdir(h5_input_dir):
    if image_name.endswith(".png"):
        # Load the PNG image
        image_path = os.path.join(h5_input_dir, image_name)
        img = Image.open(image_path)

        # Resize the image
        img_resized = img.resize((res, res))

        # Convert the resized image to a NumPy array
        img_array = np.array(img_resized)

        arr_2d = np.full((res, res), -1, dtype=int)  # Initialize the array with -1 for all values

        for target_color, value in color_map.items():
            # Create a mask of pixels that match the target_color
            mask = is_color_approx(img_array, np.array(target_color), tolerance=32)

            # Assign the corresponding value to matching pixels
            arr_2d[mask] = value

        arr_3d = np.expand_dims(arr_2d, axis=2)

        # Extract the base filename without the extension
        base_filename = os.path.splitext(os.path.basename(image_path))[0]

        # Create the new filename with the resolution
        filename = f"{base_filename}.h5"

        filepath = os.path.join(h5_output_dir, filename)

        # Create the output directory if it doesn't exist
        if not os.path.exists(h5_output_dir):
            os.makedirs(h5_output_dir)

        # Create a dataset within the 'data' group and store the array
        with h5py.File(filepath, 'w') as file:
            dset = file.create_dataset("data", data=arr_3d)

            # Add a root attribute with the name 'dx_dy_dz'
            file.attrs['dx_dy_dz'] = (0.002, 0.002, 0.002)


In [9]:
# # Create subplots to display the image and the array side by side
# fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# # Display the resized image on the left subplot
# axs[0].imshow(img_resized)
# axs[0].set_title('Resized Image')

# # Display the arr_2d array on the right subplot
# axs[1].imshow(arr_2d, cmap='viridis')  # You can change the cmap as needed
# axs[1].set_title('2D Array')

# plt.show()

In [10]:
air_in_file = f"""#python:
src_to_trunk_center = {pos[int((count-1)/2)]}
radius =  {df.loc[df['idx'] == idx, 'radius'].values[0]}

resolution = 0.002
time_window = 3e-8 #do a study
pml_cells = 25

x_gap = 0.05
y_gap = 0.02
src_to_pml = 0.05
src_to_rx = 0.1


# Derived Parameters
src_to_trunk = src_to_trunk_center-radius
diameter = radius* 2
    
pml = resolution * pml_cells
sharp_domain = [diameter, diameter + src_to_trunk]
domain = [sharp_domain[0] + pml * 2 + x_gap * 2, sharp_domain[1] +  pml * 2 + y_gap + src_to_pml]

trunk_center = [radius + pml + x_gap, src_to_trunk + radius + pml + src_to_pml]
src_position = [trunk_center[0] - (src_to_rx / 2), pml + src_to_pml, 0]
rx_position = [src_position[0] + src_to_rx, src_position[1], 0]

print('#title: Air Scan')
print("#domain: {{:.3f}} {{:.3f}} 0.002".format(domain[0], domain[1]))
print("#dx_dy_dz: {{}} {{}} {{}}".format(resolution, resolution, resolution))
print("#time_window: {{}}".format(time_window))
print()
print('#pml_cells: {{}} {{}} 0 {{}} {{}} 0'.format(pml_cells, pml_cells, pml_cells, pml_cells))
print()
print('#waveform: ricker 1 1e9 my_ricker')
print("#hertzian_dipole: z {{:.3f}} {{:.3f}} {{:.3f}} my_ricker".format(src_position[0], src_position[1], src_position[2]))
print("#rx: {{:.3f}} {{:.3f}} {{:.3f}}".format(rx_position[0], rx_position[1], rx_position[2]))
#end_python:
"""

In [11]:
in_file = f"""#python:
b_scan_cnt = {count} 
pos = {pos}
radius =  {df.loc[df['idx'] == idx, 'radius'].values[0]}

src_to_trunk_center = pos[current_model_run - 1]
resolution = 0.002
time_window = 3e-8 #do a study
pml_cells = 25

x_gap = 0.05
y_gap = 0.02
src_to_pml = 0.05
src_to_rx = 0.1


# Derived Parameters
src_to_trunk = src_to_trunk_center-radius
diameter = radius* 2
    
pml = resolution * pml_cells
sharp_domain = [diameter, diameter + src_to_trunk]
domain = [sharp_domain[0] + pml * 2 + x_gap * 2, sharp_domain[1] +  pml * 2 + y_gap + src_to_pml]

trunk_center = [radius + pml + x_gap, src_to_trunk + radius + pml + src_to_pml]
src_position = [trunk_center[0] - (src_to_rx / 2), pml + src_to_pml, 0]
rx_position = [src_position[0] + src_to_rx, src_position[1], 0]

print('#title: Rotating Straight Scan')
print("#domain: {{:.3f}} {{:.3f}} 0.002".format(domain[0], domain[1]))
print("#dx_dy_dz: {{}} {{}} {{}}".format(resolution, resolution, resolution))
print("#time_window: {{}}".format(time_window))
print()
print('#pml_cells: {{}} {{}} 0 {{}} {{}} 0'.format(pml_cells, pml_cells, pml_cells, pml_cells))
print()
print("#geometry_objects_read: {{:.3f}} {{:.3f}} 0 h5/{mode}{idx}_{{}}.h5 materials.txt".format((trunk_center[0]) - radius, (trunk_center[1]) - radius, current_model_run - 1)) 
print()
print('#waveform: ricker 1 1e9 my_ricker')
print("#hertzian_dipole: z {{:.3f}} {{:.3f}} {{:.3f}} my_ricker".format(src_position[0], src_position[1], src_position[2]))
print("#rx: {{:.3f}} {{:.3f}} {{:.3f}}".format(rx_position[0], rx_position[1], rx_position[2]))
if (current_model_run == (b_scan_cnt - 1)/2):
       print("#geometry_objects_write: {{:.3f}} {{:.3f}} 0 {{:.3f}} {{:.3f}} 0.002 {mode}".format(trunk_center[0] - radius, trunk_center[1] - src_to_trunk_center, trunk_center[0] + radius, trunk_center[1] + radius))
#end_python:
"""

In [12]:
material_file = f"""
#material: {df.loc[df['idx'] == idx, 'eps_bark'].values[0]} 0 1 0 bark
#material: {df.loc[df['idx'] == idx, 'eps_sapwood'].values[0]} 0 1 0 sapwood
#material: {df.loc[df['idx'] == idx, 'eps_heartwood'].values[0]} 0 1 0 heartwood
#material: 1 0 1 0 cavity
"""

In [13]:
# Specify the file path where you want to save the Python code
in_path = f"{working_dir}/{mode}.in"

# Open the file in write mode
with open(in_path, "w") as file:
    # Write the Python code to the file
    file.write(in_file)

# Specify the file path where you want to save the Python code
mat_path = f"{working_dir}/materials.txt"

# Open the file in write mode
with open(mat_path, "w") as file:
    # Write the Python code to the file
    file.write(material_file)

# Specify the file path where you want to save the Python code
air_path = f"{working_dir}/air.in"

# Open the file in write mode
with open(air_path, "w") as file:
    # Write the Python code to the file
    file.write(air_in_file)

In [14]:
api(air_path, n= 1, geometry_only=False, geometry_fixed=False)


=== Electromagnetic modelling software based on the Finite-Difference Time-Domain (FDTD) method 

    www.gprmax.com   __  __
     __ _ _ __  _ __|  \/  | __ ___  __
    / _` | '_ \| '__| |\/| |/ _` \ \/ /
   | (_| | |_) | |  | |  | | (_| |>  <
    \__, | .__/|_|  |_|  |_|\__,_/_/\_\
    |___/|_|
                     v3.1.6 (Big Smoke)

 Copyright (C) 2015-2023: The University of Edinburgh
 Authors: Craig Warren and Antonis Giannopoulos

 gprMax is free software: you can redistribute it and/or modify it under the
  terms of the GNU General Public License as published by the Free Software
  Foundation, either version 3 of the License, or (at your option) any later
  version.
 gprMax is distributed in the hope that it will be useful, but WITHOUT ANY
  WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR
  A PARTICULAR PURPOSE.  See the GNU General Public License for more details.
 You should have received a copy of the GNU General Public License along with
  gpr

In [15]:
api(in_path, n= count, geometry_only=False, geometry_fixed=False)

In [16]:
merge_files(f'{working_dir}/{mode}',True)

# Delete png rotated files
files = glob.glob(os.path.join(png_output_dir, '*.png'))
# Iterate over the list of .txt files and delete them
for file in files:
    try:
        os.remove(file)
    except Exception as e:
        print(f"Error deleting {file}: {e}")

# Delete h5 rotated files
files = glob.glob(os.path.join(h5_output_dir, '*.h5'))
# Iterate over the list of .txt files and delete them
for file in files:
    try:
        os.remove(file)
    except Exception as e:
        print(f"Error deleting {file}: {e}")

In [27]:
with h5py.File(f'{working_dir}/air.out', 'r') as f:
    Ez0 =  f['rxs']['rx1']['Ez'][()]

Ez0 = Ez0[:, np.newaxis]  # Add a new axis

In [21]:
with h5py.File(f'{working_dir}/{mode}.h5', 'r') as f2:
    dset = f2['data'][()]

# Set the desired width and height of the figure
fig_width = dset.shape[1] * 0.01
fig_height = dset.shape[0] * 0.01

# Create the figure and axes objects with the desired size and arrangement
fig, ax = plt.subplots(figsize=(fig_width, fig_height))

# Plot dset on the left subplot
## For 3D
# axs[0].imshow(np.transpose(dset[310,:,:], axes=(1, 0)), cmap='viridis')
ax.imshow(dset, cmap='viridis')
ax.invert_yaxis()

# Display the image
plt.axis('off')
plt.savefig(f"{output_dir}/geometry/{mode}{idx}.png", format='png', dpi=300, bbox_inches='tight', pad_inches=0)
plt.close()

In [22]:
with h5py.File(f'{working_dir}/{mode}_merged.out', 'r') as f:
    bscan =  np.subtract(f['rxs']['rx1']['Ez'][()], Ez0)

# Set the desired width and height of the figure
fig_width = 2
fig_height = 5

# Create the figure and axes objects with the desired size and arrangement
fig, ax = plt.subplots(figsize=(fig_width, fig_height))

# Plot dset on the left subplot
## For 3D
# axs[0].imshow(np.transpose(dset[310,:,:], axes=(1, 0)), cmap='viridis')
ax.imshow(bscan, cmap='gray', aspect = 'auto')

# Display the image
plt.axis('off')
plt.savefig(f"{output_dir}/png/{mode}{idx}_bscan.png", format='png', dpi=300, bbox_inches='tight', pad_inches=0)
plt.close()

In [ ]:
# Copy the source file to the destination
shutil.copy(f'{working_dir}/{mode}_merged.out', f'{output_dir}/out/{mode}{idx}.out')

'output/out/defect0.out'